<a href="https://colab.research.google.com/github/HoangHungLN/MachineLearning_Assignment/blob/main/Extended_Assignment/notebooks/Extended_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import json
import os
import getpass, os, subprocess, textwrap


os.makedirs("data", exist_ok = True)
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json -O data/train.json
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json -O data/dev.json

DATA_FILE = 'data/train.json'
DEV_FILE = 'data/dev.json'
PARAM_DIR = 'Extended_Assigment/parameters'

--2025-11-10 04:28:07--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27164412 (26M) [text/plain]
Saving to: ‘data/train.json’

data/train.json     100%[===================>]  25.91M  --.-KB/s    in 0.1s    

2025-11-10 04:28:08 (208 MB/s) - ‘data/train.json’ saved [27164412/27164412]

--2025-11-10 04:28:08--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133

In [8]:
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    raw = json.load(f)

def build_sentences(raw, use='chunk'):  # use: 'chunk', 'pos', hoặc 'none'
    out = []
    for ex in raw:
        if not isinstance(ex, dict) or 'sentence' not in ex:
            continue
        words = ex['sentence']

        tags = None
        if use != 'none':
            # ưu tiên đúng khóa nếu có
            pref = {'chunk': 'chunk_tags', 'pos': 'pos'}[use]
            tags = ex.get(pref) or ex.get('tags') or ex.get('labels')

        if isinstance(tags, list) and len(tags) == len(words):
            out.append(list(zip(words, tags)))   # [(word, tag), ...]
        else:
            out.append(words)                    # chỉ word nếu không có tag
    return out

sentences = build_sentences(raw, use='chunk')   # đổi 'pos' hoặc 'none' tùy bài
print("Số câu xử lý được:", len(sentences))
print("Ví dụ câu đầu:", sentences[0][:10])

Số câu xử lý được: 38218
Ví dụ câu đầu: [('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT')]


In [11]:
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    raw_train = json.load(f)
with open(DEV_FILE, 'r', encoding='utf-8') as f:
    raw_dev = json.load(f)

def detect_tag_key(ex):
    for k in ['pos','upos','tags','labels']:
        if k in ex and isinstance(ex[k], list) and len(ex[k]) == len(ex['sentence']):
            return k
    raise ValueError(f"Không tìm thấy trường nhãn song song với 'sentence'. Keys: {list(ex.keys())}")

TAG_KEY = detect_tag_key(raw_train[0])
print("TAG_KEY =", TAG_KEY)

X_train = [ex['sentence'] for ex in raw_train]
Y_train = [ex[TAG_KEY]     for ex in raw_train]
X_dev   = [ex['sentence'] for ex in raw_dev]
Y_dev   = [ex[TAG_KEY]     for ex in raw_dev]
print(len(X_train), "câu train |", len(X_dev), "câu dev")

TAG_KEY = labels
38218 câu train | 5527 câu dev


In [14]:
UNK = "<UNK>"
min_freq = 1  # muốn “mạnh tay” hơn thì tăng lên 2,3,...

word_freq = Counter(w for sent in X_train for w in sent)
vocab = {w for w,c in word_freq.items() if c > min_freq}
vocab.add(UNK)
vocab = sorted(vocab)

tag_set = sorted({t for tags in Y_train for t in tags})

word2id = {w:i for i,w in enumerate(vocab)}
tag2id  = {t:i for i,t in enumerate(tag_set)}
id2tag  = {i:t for t,i in tag2id.items()}

def map_unk(sent):
    return [w if w in word2id else UNK for w in sent]

X_train = [map_unk(s) for s in X_train]
X_dev   = [map_unk(s) for s in X_dev]

len(vocab), len(tag_set), X_train[0][:10], Y_train[0][:10]


(23183,
 45,
 ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the'],
 ['NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT'])

In [19]:
import numpy as np

# YÊU CẦU: đã có X_train, Y_train, tag2id, word2id trong bộ nhớ.

def estimate_hmm_supervised(X, Y, tag2id, word2id,
                            alpha_pi=1.0, alpha_A=1.0, alpha_B=1e-3):
    K, V = len(tag2id), len(word2id)
    pi_cnt = np.full(K,     alpha_pi, dtype=np.float64)
    A_cnt  = np.full((K,K), alpha_A,  dtype=np.float64)
    B_cnt  = np.full((K,V), alpha_B,  dtype=np.float64)

    for words, tags in zip(X, Y):
        if not words:
            continue
        pi_cnt[tag2id[tags[0]]] += 1
        for t in range(len(words)):
            j = tag2id[tags[t]]
            w = word2id[words[t]]
            B_cnt[j, w] += 1
            if t < len(words)-1:
                i = tag2id[tags[t]]
                k = tag2id[tags[t+1]]
                A_cnt[i, k] += 1

    pi = pi_cnt / pi_cnt.sum()
    A  = A_cnt / A_cnt.sum(axis=1, keepdims=True)
    B  = B_cnt / B_cnt.sum(axis=1, keepdims=True)
    return pi, A, B, np.log(pi), np.log(A), np.log(B)

# ƯỚC LƯỢNG
pi, A, B, log_pi, log_A, log_B = estimate_hmm_supervised(X_train, Y_train, tag2id, word2id)

# KIỂM TRA
print("sum(pi) =", float(pi.sum()))
print("row sums A (5 mẫu) =", np.round(A.sum(axis=1)[:5], 6).tolist())
print("row sums B (5 mẫu) =", np.round(B.sum(axis=1)[:5], 6).tolist())


sum(pi) = 1.0000000000000002
row sums A (5 mẫu) = [1.0, 1.0, 1.0, 1.0, 1.0]
row sums B (5 mẫu) = [1.0, 1.0, 1.0, 1.0, 1.0]


In [20]:
os.makedirs(PARAM_DIR, exist_ok=True)

# 1) Tham số HMM
np.savez_compressed(
    os.path.join(PARAM_DIR, "hmm_params_supervised.npz"),
    pi=pi, A=A, B=B,
    log_pi=log_pi, log_A=log_A, log_B=log_B
)

# 2) Ánh xạ và cấu hình để teammate load lại đúng thứ tự
config = {
    "UNK": UNK,
    "tag_key": TAG_KEY,
    "vocab":   vocab,      # giữ nguyên thứ tự tương ứng với ma trận B
    "tag_set": tag_set,    # giữ nguyên thứ tự tương ứng với các ma trận
    "alpha": {"pi":1.0, "A":1.0, "B":1e-3}
}
with open(os.path.join(PARAM_DIR, "tag_word_maps.json"), "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print("Saved files:", os.listdir(PARAM_DIR))

Saved files: ['hmm_params_supervised.npz', 'tag_word_maps.json']
